# Задание сделать парсер для сбора корпуса текста

In [ ]:
%pip install fake_useragent
%pip install bs4
%pip install pymorphy2
%pip install spacy
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=830a266ff6b8af2fa5e612655a008f7fe79156ebda0e03702afa2661a56d07b9
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [ ]:
import warnings
warnings.filterwarnings("ignore")  # Отключаем все предупреждения Python (чтобы не засорять вывод при работе с большими данными)

# ========================================
# БИБЛИОТЕКИ
# ========================================
import requests  # Для выполнения HTTP-запросов к веб-страницам
from fake_useragent import UserAgent  # Генерирует случайные User-Agent, чтобы сайты не блокировали нас как бота
from bs4 import BeautifulSoup  # Парсинг HTML-страниц — извлечение текста, заголовков, метаданных
import time  # Задержки между запросами (чтобы не перегружать серверы и избежать бана)
import random  # Случайные задержки и выбор поисковых систем
import re  # Регулярные выражения — для очистки текста, поиска авторов, фильтрации
import string  # Для работы с символами пунктуации
import pandas as pd  # Сохранение данных в CSV-таблицу
import os  # Работа с файловой системой (не используется напрямую, но может пригодиться)
from urllib.parse import urljoin, urlparse, quote_plus  # Работа с URL: сборка, разбор, кодирование поисковых запросов
from datetime import datetime  # Работа с датами (извлечение даты публикации)
import nltk  # Библиотека для обработки естественного языка (токенизация, стоп-слова)
from nltk import word_tokenize  # Разбиение текста на слова (токены)
from nltk.corpus import stopwords  # Список стоп-слов (предлоги, союзы — не несут смысла)
import pymorphy2  # Лемматизация русских слов (приведение к начальной форме: "бегу" → "бежать")
import inspect  # Используется для патча pymorphy2 (внутренняя совместимость)

print("Инициализация библиотек...")  # Уведомление о начале загрузки зависимостей

# ========================================
# ПАТЧ ДЛЯ PYMORPHY2
# ========================================
def patch_pymorphy2():
    """
    Исправление ошибки совместимости pymorphy2 с новыми версиями Python.
    В новых версиях inspect.getargspec устарел → заменяем на getfullargspec.
    Это нужно, чтобы pymorphy2 корректно анализировал функции.
    """
    def getargspec_patch(func):
        try:
            # Получаем полную информацию о параметрах функции
            args, varargs, varkw, defaults, kwonlyargs, kwonlydefaults, annotations = inspect.getfullargspec(func)
            return args, varargs, varkw, defaults  # Возвращаем только первые 4 — как ожидает старая версия
        except Exception:
            return [], None, None, None  # Если ошибка — возвращаем безопасные значения
    # Заменяем старую функцию на нашу исправленную
    inspect.getargspec = getargspec_patch

patch_pymorphy2()  # Применяем патч сразу после импорта

# ========================================
# НАСТРОЙКА NLTK
# ========================================
try:
    # Проверяем, загружены ли необходимые ресурсы NLTK
    nltk.data.find('tokenizers/punkt')  # Токенизатор для разделения на предложения
    nltk.data.find('tokenizers/punkt_tab')  # Дополнительный токенизатор
    nltk.data.find('corpora/stopwords')  # Стоп-слова
except LookupError:
    print("Загрузка ресурсов NLTK...")
    # Если не загружены — скачиваем автоматически
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
print("  ✅ NLTK готов")  # Подтверждение успешной инициализации

# ========================================
# НАСТРОЙКА PYMORPHY2
# ========================================
morph = pymorphy2.MorphAnalyzer()  # Создаём объект для морфологического анализа русского языка
print("  ✅ Pymorphy2 готов")

# ========================================
# СТОП-СЛОВА
# ========================================
# Объединяем русские и английские стоп-слова в один набор (set — для быстрого поиска)
stop_words = set(stopwords.words('russian') + stopwords.words('english'))
print("  ✅ Стоп-слова загружены")

# ========================================
# РАСШИРЕННЫЙ СПИСОК ИГР И ТЕМ (более 100 элементов)
# ========================================
GAMES = [
    # === РУССКОЯЗЫЧНЫЕ ИГРЫ ===
    "Atomic Heart", "Metro Exodus", "Escape from Tarkov", "Pathfinder",
    "S.T.A.L.K.E.R.", "War Thunder", "World of Tanks", "Crossout",

    # === ИНДИ-ИГРЫ ===
    "Hollow Knight", "Hollow Knight Silksong", "Platypus",
    "Hard Truck Apocalypse", "No Man's Sky", "Moonlighter", "Minecraft",
    "Celeste", "Hades", "Stardew Valley", "Undertale", "Terraria",
    "Dead Cells", "Ori and the Blind Forest", "Cuphead", "Shovel Knight",
    "Subnautica", "The Binding of Isaac", "Risk of Rain 2", "Slay the Spire",

    # === AAA-ИГРЫ ===
    "The Witcher 3", "Elden Ring", "Dark Souls", "Sekiro", "Bloodborne",
    "God of War", "Red Dead Redemption 2", "Cyberpunk 2077", "Skyrim",
    "Grand Theft Auto V", "The Last of Us", "Horizon Zero Dawn",

    # === СТРАТЕГИИ ===
    "Civilization VI", "StarCraft II", "Age of Empires IV", "Total War",
    "XCOM 2", "Crusader Kings III", "Cities Skylines",

    # === ШУТЕРЫ ===
    "Counter-Strike 2", "Valorant", "Apex Legends", "Overwatch 2",
    "Call of Duty", "Battlefield", "Destiny 2", "Halo Infinite",

    # === ММОРПГ / ОНЛАЙН ===
    "World of Warcraft", "Final Fantasy XIV", "Guild Wars 2", "Dota 2", "League of Legends",

    # === ОБЩИЕ ТЕМЫ (не игры, но связанные с индустрией) ===
    "обзор игры", "прохождение игры", "рецензия игры", "инди игры",
    "киберспорт", "геймдизайн", "игровая индустрия", "стримы",
    "мобильные игры", "консольные игры", "PC игры", "VR игры",

    # === ДОПОЛНИТЕЛЬНЫЕ ТЕМЫ (для расширения корпуса) ===
    "искусственный интеллект", "машинное обучение", "программирование",
    "веб-разработка", "мобильные приложения", "кибербезопасность",
    "блокчейн технологии", "облачные вычисления", "нейросети",
    "интернет вещей", "big data", "devops", "frontend", "backend",
    "космические исследования", "физика частиц", "квантовая механика",
    "биотехнологии", "генетика", "экология", "астрономия",
    "современное искусство", "кинематограф", "литература",
    "образование в России", "психология", "социология",
    "новости науки", "технологии 2024", "стартапы России"
]

# ========================================
# ФУНКЦИЯ ИЗВЛЕЧЕНИЯ АВТОРА
# ========================================
def extract_author(soup, url):
    """
    Извлекает имя автора статьи из HTML-кода страницы.
    Использует несколько стратегий:
    1. Мета-теги (наиболее надёжный способ)
    2. Классы с ключевыми словами (author, byline и т.д.)
    3. Специфичные CSS-селекторы
    4. Если ничего не найдено — возвращает домен сайта
    """
    # --- Попытка 1: Мета-теги ---
    meta_tags = [
        soup.find('meta', {'name': 'author'}),
        soup.find('meta', {'property': 'article:author'}),
        soup.find('meta', {'name': 'article:author'}),
        soup.find('meta', {'property': 'author'}),
    ]

    for tag in meta_tags:
        if tag and tag.get('content'):
            return tag.get('content').strip()  # Возвращаем первый найденный

    # --- Попытка 2: Классы с ключевыми словами ---
    author_classes = [
        'author', 'author-name', 'post-author', 'article-author',
        'byline', 'by-author', 'entry-author', 'writer',
        'автор', 'author_name', 'article__author'
    ]

    for class_name in author_classes:
        author_tag = soup.find(class_=re.compile(class_name, re.I))  # re.I — игнор регистра
        if author_tag:
            text = author_tag.get_text(strip=True)
            # Убираем префиксы вроде "by ", "автор: "
            text = re.sub(r'^(by|автор|written by|posted by)[\s:]+', '', text, flags=re.I)
            if text and len(text) < 100:  # Защита от слишком длинного текста
                return text

    # --- Попытка 3: CSS-селекторы ---
    selectors = [
        'a[rel="author"]',
        'span[itemprop="author"]',
        'span[itemprop="name"]',
        '[class*="author"] a',
        '.author-info a',
    ]

    for selector in selectors:
        author_tag = soup.select_one(selector)
        if author_tag:
            text = author_tag.get_text(strip=True)
            if text and len(text) < 100:
                return text

    # --- Запасной вариант: домен сайта ---
    domain = urlparse(url).netloc
    return domain.replace('www.', '').split('.')[0].capitalize()

# ========================================
# ПОИСК ЧЕРЕЗ YANDEX (для русскоязычного контента)
# ========================================
def search_yandex(query, num_results=15):
    """
    Ищет в Яндексе по запросу и возвращает список ссылок.
    Использует прямой запрос к странице поиска.
    Парсит результаты с классами 'Link.*Organic'.
    """
    try:
        search_url = f"https://yandex.ru/search/?text={quote_plus(query)}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept-Language': 'ru-RU,ru;q=0.9'
        }
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = []
        # Основной способ: органические результаты
        for result in soup.find_all('a', class_=re.compile('Link.*Organic'), limit=num_results):
            href = result.get('href')
            if href and href.startswith('http'):
                links.append(href)

        # Резервный способ: через <li class="serp-item">
        if not links:
            for result in soup.find_all('li', class_=re.compile('serp-item'), limit=num_results):
                link = result.find('a')
                if link and link.get('href'):
                    href = link.get('href')
                    if href.startswith('http'):
                        links.append(href)

        print(f"    🔍 Yandex: найдено {len(links)} ссылок для '{query}'")
        return links
    except Exception as e:
        print(f"    ⚠ Ошибка поиска Yandex: {e}")
        return []

# ========================================
# ПОИСК ЧЕРЕЗ DUCKDUCKGO
# ========================================
def search_duckduckgo(query, num_results=15):
    """
    Поиск через DuckDuckGo (HTML-версия, без JavaScript).
    Полезен как резервный источник, особенно для международного контента.
    """
    try:
        search_url = f"https://html.duckduckgo.com/html/?q={quote_plus(query)}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept-Language': 'ru-RU,ru;q=0.9,en;q=0.8'
        }
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = []
        for result in soup.find_all('a', class_='result__a', limit=num_results):
            href = result.get('href')
            if href and href.startswith('http'):
                links.append(href)

        print(f"    🔍 DuckDuckGo: найдено {len(links)} ссылок для '{query}'")
        return links
    except Exception as e:
        print(f"    ⚠ Ошибка поиска DuckDuckGo: {e}")
        return []

# ========================================
# СБОР БАЗОВЫХ URL (статичные источники)
# ========================================
def get_base_urls():
    """
    Создаёт список прямых ссылок на игровые сайты.
    Использует шаблоны поиска для популярных русскоязычных и международных ресурсов.
    """
    base_urls = []

    # Русскоязычные игровые сайты
    ru_templates = [
        "https://stopgame.ru/search?q={game}",
        "https://dtf.ru/search?q={game}",
        "https://igromania.ru/search/?q={game}",
        "https://www.playground.ru/search?q={game}",
        "https://www.kanobu.ru/search/?q={game}",
        "https://vgtimes.ru/search/?q={game}",
        "https://gamemag.ru/search?q={game}",
        "https://cybersport.ru/search?q={game}",
    ]

    # Международные источники
    en_templates = [
        "https://ru.wikipedia.org/wiki/{game}",
        "https://store.steampowered.com/search/?term={game}",
        "https://www.metacritic.com/search/{game}",
        "https://www.ign.com/games/{game}",
        "https://www.pcgamer.com/search/?searchTerm={game}",
        "https://www.gamespot.com/search/?q={game}",
    ]

    all_templates = ru_templates + en_templates

    for game in GAMES:
        game_slug = game.lower().replace(' ', '-').replace("'", '')  # Для URL вроде /elden-ring
        game_encoded = quote_plus(game)  # Для поисковых запросов

        for template in all_templates:
            if '{game}' in template:
                if 'search' in template or '?' in template:
                    url = template.format(game=game_encoded)
                else:
                    url = template.format(game=game_slug)
                base_urls.append(url)

    return base_urls

# ========================================
# ГЛАВНАЯ ФУНКЦИЯ СБОРА URL
# ========================================
def collect_urls(target_count=1500):
    """
    Собирает уникальные URL из:
    1. Базовых шаблонов
    2. Поиска в Yandex и DuckDuckGo
    3. Фильтрует соцсети и приоритет отдает русскоязычным сайтам
    """
    print(f"\n📡 Сбор URL (цель: {target_count})...")
    all_urls = set()  # Уникальные URL

    # --- 1. Базовые URL ---
    print("\n1️⃣ Добавление базовых URL...")
    base_urls = get_base_urls()
    all_urls.update(base_urls)
    print(f"  ✅ Базовых URL: {len(all_urls)}")

    # --- 2. Поиск через поисковые системы ---
    print("\n2️⃣ Поиск через Yandex & DuckDuckGo...")
    search_queries = []

    # Поисковые запросы по играм
    for game in GAMES[:40]:
        search_queries.extend([
            f"{game} обзор игры",
            f"{game} прохождение",
            f"{game} рецензия",
            f"{game} статья",
        ])

    # Общие темы
    general_topics = [
        "обзоры инди игр", "лучшие игры 2024", "игровые новости",
        "разработка игр", "киберспорт", "игровая культура",
        "ретро игры", "геймдизайн", "история видеоигр",
        "игровая индустрия", "игровая механика", "РПГ игры"
    ]
    search_queries.extend(general_topics)

    # --- Цикл поиска ---
    for i, query in enumerate(search_queries[:800]):
        if len(all_urls) >= target_count:
            break

        print(f"  🔍 Запрос {i+1}/800: '{query}'")

        # 70% запросов через Yandex (лучше для русского)
        if random.random() > 0.3:
            yandex_links = search_yandex(query, num_results=500)
            all_urls.update(yandex_links)
            time.sleep(random.uniform(2, 4))

        # Остальные — через DuckDuckGo
        if len(all_urls) < target_count:
            ddg_links = search_duckduckgo(query, num_results=500)
            all_urls.update(ddg_links)
            time.sleep(random.uniform(2, 4))

        if i % 10 == 0:
            print(f"  📊 Всего собрано URL: {len(all_urls)}")

    # --- 3. Фильтрация ---
    filtered_urls = []
    blocked_domains = ['youtube.com', 'twitter.com', 'facebook.com', 'vk.com', 't.me']  # Соцсети
    ru_priority_domains = ['stopgame.ru', 'dtf.ru', 'igromania.ru', 'habr.com']  # Приоритет

    ru_urls = []
    other_urls = []

    for url in all_urls:
        domain = urlparse(url).netloc
        if any(blocked in domain for blocked in blocked_domains):
            continue
        if any(ru_domain in domain for ru_domain in ru_priority_domains):
            ru_urls.append(url)
        else:
            other_urls.append(url)

    filtered_urls = ru_urls + other_urls  # Сначала русские

    print(f"\n✅ Итоговое количество URL: {len(filtered_urls)}")
    print(f"   📌 Русскоязычных: {len(ru_urls)}")
    print(f"   🌐 Остальных: {len(other_urls)}")

    return filtered_urls[:target_count]

# ========================================
# HTTP SESSION С СЛУЧАЙНЫМ USER-AGENT
# ========================================
def create_http_session():
    """
    Создаёт сессию requests с случайным User-Agent.
    Это помогает обойти защиту от ботов.
    """
    session = requests.Session()
    ua = UserAgent()
    session.headers.update({
        'User-Agent': ua.random,
        'Accept-Language': 'ru-RU,ru;q=0.9,en;q=0.8'
    })
    return session

# ========================================
# ПАРСИНГ СТРАНИЦЫ
# ========================================
def parse_page(url, session, max_retries=3):
    """
    Загружает страницу, извлекает:
    - Заголовок
    - Автор
    - Основной текст
    - Дату публикации
    Удаляет скрипты, стили, навигацию.
    """
    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=15)
            response.raise_for_status()

            # Исправление кодировки
            if response.encoding.lower() in ['iso-8859-1', 'windows-1251']:
                response.encoding = response.apparent_encoding

            soup = BeautifulSoup(response.content, 'html.parser')

            # Удаление ненужных элементов
            for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside', 'iframe']):
                tag.decompose()

            # Заголовок
            title = soup.title.string.strip() if soup.title else 'Без заголовка'

            # Автор
            author = extract_author(soup, url)

            # Текст (параграфы, статьи и т.д.)
            content = soup.find_all(['p', 'article', 'div', 'section'], limit=150)
            raw_text = ' '.join([elem.get_text(strip=True) for elem in content if elem.get_text(strip=True)])

            if not raw_text or len(raw_text) < 200:
                return None

            # Дата
            date = datetime.now().strftime('%Y-%m-%d')
            date_meta = soup.find('meta', {'property': 'article:published_time'}) or \
                       soup.find('meta', {'name': 'date'}) or \
                       soup.find('time')

            if date_meta:
                date_str = date_meta.get('content') or date_meta.get('datetime') or date_meta.get_text()
                try:
                    date = datetime.fromisoformat(date_str.split('T')[0]).strftime('%Y-%m-%d')
                except:
                    pass

            return {
                'title': title,
                'author': author,
                'raw_text': raw_text[:10000],  # Ограничиваем размер
                'date': date
            }
        except (requests.RequestException, Exception) as e:
            if attempt < max_retries - 1:
                time.sleep(random.uniform(1, 3))
    return None

# ========================================
# ОЧИСТКА И ЛЕММАТИЗАЦИЯ ТЕКСТА
# ========================================
def clean_text(raw_text):
    """
    1. Удаляет HTML, цифры, пунктуацию
    2. Приводит к нижнему регистру
    3. Токенизирует
    4. Убирает стоп-слова
    5. Лемматизирует с помощью pymorphy2
    """
    text = re.sub(r'<[^>]+>', ' ', raw_text)
    text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)
    text = re.sub(r'\d+\.?\d*', ' ', text)
    text = re.sub(r'[{}]'.format(string.punctuation + '«»—–…""„'), ' ', text)
    text = re.sub(r'\s+', ' ', text.strip())

    tokens = word_tokenize(text.lower())

    lemmatized_tokens = []
    for token in tokens:
        if token.isalpha() and len(token) > 2 and token not in stop_words:
            parsed = morph.parse(token)[0]
            lemmatized_tokens.append(parsed.normal_form)

    return ' '.join(lemmatized_tokens[:400]), len(lemmatized_tokens[:400])

# ========================================
# ОПРЕДЕЛЕНИЕ ИГРЫ ПО URL И ЗАГОЛОВКУ
# ========================================
def get_game_from_url(url, title):
    """
    Пытается определить, о какой игре статья.
    Ищет название игры в URL или заголовке.
    """
    text_to_check = (url + ' ' + title).lower()
    for game in GAMES:
        if game.lower().replace(' ', '') in text_to_check.replace(' ', '').replace('-', ''):
            return game
    return "Игры (общее)"

# ========================================
# ОСНОВНОЙ ПРОЦЕСС
# ========================================
def main():
    print("\n" + "="*70)
    print("🎮 СБОРЩИК ТЕКСТОВОГО КОРПУСА - 1000+ ДОКУМЕНТОВ (РУССКОЯЗЫЧНЫЙ)")
    print("="*70)

    # --- Параметры ---
    target_docs = 1000
    target_urls = 1800
    unique_urls = collect_urls(target_count=target_urls)

    print(f"\n🚀 Начало парсинга {len(unique_urls)} URL (цель: {target_docs} документов)...")
    session = create_http_session()
    corpus_data = []
    parsed_urls = set()
    doc_id = 1

    random.shuffle(unique_urls)  # Перемешиваем, чтобы избежать блокировок

    for i, url in enumerate(unique_urls):
        if doc_id > target_docs:
            break
        if url in parsed_urls:
            continue

        if i % 50 == 0:
            print(f"\n📊 Прогресс: {i}/{len(unique_urls)} URL | {doc_id-1} документов собрано")

        print(f"  🔍 [{doc_id}] {url[:80]}...")
        parsed = parse_page(url, session)

        if parsed and parsed['raw_text']:
            cleaned_text, token_count = clean_text(parsed['raw_text'])

            if token_count > 30:
                game = get_game_from_url(url, parsed['title'])
                corpus_data.append({
                    'doc_id': doc_id,
                    'game': game,
                    'title': parsed['title'][:200],
                    'author': parsed['author'],
                    'url': url,
                    'raw_text': parsed['raw_text'][:2000],
                    'cleaned_text': cleaned_text,
                    'tokens_count': token_count,
                    'date': parsed['date']
                })
                parsed_urls.add(url)
                print(f"    ✅ Документ {doc_id}: {token_count} токенов | Автор: {parsed['author']} | {game}")
                doc_id += 1
            else:
                print(f"    ⚠ Пропущено: мало текста ({token_count} токенов)")
        else:
            print(f"    ⚠ Пропущено: ошибка парсинга")

        time.sleep(random.uniform(0.5, 2.5))  # Вежливая задержка

    # --- СОХРАНЕНИЕ В TXT ---
    print(f"\n💾 Создание корпуса: game_corpus_{len(corpus_data)}.txt")
    with open(f'game_corpus_{len(corpus_data)}.txt', 'w', encoding='utf-8') as f:
        for doc in corpus_data:
            f.write(f"{'='*80}\n")
            f.write(f"ДОКУМЕНТ {doc['doc_id']} | {doc['game']}\n")
            f.write(f"{'='*80}\n")
            f.write(f"Заголовок: {doc['title']}\n")
            f.write(f"Автор: {doc['author']}\n")
            f.write(f"Дата: {doc['date']}\n")
            f.write(f"Токенов: {doc['tokens_count']}\n")
            f.write(f"URL: {doc['url']}\n")
            f.write(f"{'-'*80}\n")
            f.write(f"{doc['cleaned_text']}\n")
            f.write(f"{'='*80}\n\n")
    print(f"  ✅ TXT корпус сохранён: {len(corpus_data)} документов")

    # --- СОХРАНЕНИЕ В CSV ---
    df = pd.DataFrame(corpus_data)
    df.to_csv(f'game_corpus_{len(corpus_data)}.csv', index=False, encoding='utf-8-sig')
    print(f"  ✅ CSV сохранён: game_corpus_{len(corpus_data)}.csv")

    # --- СТАТИСТИКА ---
    total_tokens = sum(doc['tokens_count'] for doc in corpus_data)
    game_counts = df['game'].value_counts()
    author_counts = df['author'].value_counts()

    print(f"\n{'='*70}")
    print(f"📊 СТАТИСТИКА КОРПУСА")
    print(f"{'='*70}")
    print(f"  📄 Документов: {len(corpus_data)}")
    print(f"  🔤 Всего токенов: {total_tokens:,}")
    print(f"  📈 Средн. токенов/документ: {total_tokens / len(corpus_data):.2f}" if corpus_data else "  Нет данных")
    print(f"  🔗 Уникальных URL: {len(parsed_urls)}")
    print(f"  ✍️  Уникальных авторов: {len(author_counts)}")

    print(f"\n🎮 Топ-10 игр по количеству документов:")
    for game, count in game_counts.head(10).items():
        print(f"    {game}: {count} документов")

    print(f"\n✍️  Топ-10 авторов:")
    for author, count in author_counts.head(10).items():
        print(f"    {author}: {count} документов")

    print(f"\n🎉 Корпус готов для NLP-анализа (LDA, кластеризация, тематическое моделирование)!")
    print(f"📁 Файлы: game_corpus_{len(corpus_data)}.txt, game_corpus_{len(corpus_data)}.csv")

# ========================================
# ЗАПУСК
# ========================================
if __name__ == "__main__":
    main()  # Запускаем основную функцию при выполнении скрипта

Инициализация библиотек...
Загрузка ресурсов NLTK...
  ✅ NLTK готов
  ✅ Pymorphy2 готов
  ✅ Стоп-слова загружены

🎮 СБОРЩИК ТЕКСТОВОГО КОРПУСА - 1000+ ДОКУМЕНТОВ (РУССКОЯЗЫЧНЫЙ)

📡 Сбор URL (цель: 1800)...

1️⃣ Добавление базовых URL...
  ✅ Базовых URL: 1428

2️⃣ Поиск через Yandex & DuckDuckGo...
  🔍 Запрос 1/800: 'Atomic Heart обзор игры'
    🔍 DuckDuckGo: найдено 0 ссылок для 'Atomic Heart обзор игры'
  📊 Всего собрано URL: 1428
  🔍 Запрос 2/800: 'Atomic Heart прохождение'
    🔍 DuckDuckGo: найдено 0 ссылок для 'Atomic Heart прохождение'
  🔍 Запрос 3/800: 'Atomic Heart рецензия'
    🔍 DuckDuckGo: найдено 0 ссылок для 'Atomic Heart рецензия'
  🔍 Запрос 4/800: 'Atomic Heart статья'
    🔍 Yandex: найдено 0 ссылок для 'Atomic Heart статья'
    🔍 DuckDuckGo: найдено 0 ссылок для 'Atomic Heart статья'
  🔍 Запрос 5/800: 'Metro Exodus обзор игры'
    🔍 Yandex: найдено 0 ссылок для 'Metro Exodus обзор игры'
    🔍 DuckDuckGo: найдено 0 ссылок для 'Metro Exodus обзор игры'
  🔍 Запрос 6/800: 'M

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# БИБЛИОТЕКИ
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
import random
import re
import string
import pandas as pd
import os
from urllib.parse import urljoin, urlparse, quote_plus
from datetime import datetime
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import inspect

print("🔧 Инициализация библиотек...")

# ПАТЧ ДЛЯ PYMORPHY2
def patch_pymorphy2():
    def getargspec_patch(func):
        try:
            args, varargs, varkw, defaults, kwonlyargs, kwonlydefaults, annotations = inspect.getfullargspec(func)
            return args, varargs, varkw, defaults
        except Exception:
            return [], None, None, None
    inspect.getargspec = getargspec_patch

patch_pymorphy2()

# НАСТРОЙКА NLTK
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('tokenizers/punkt_tab')
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Загрузка ресурсов NLTK...")
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
print("  ✅ NLTK готов")

# НАСТРОЙКА PYMORPHY2
morph = pymorphy2.MorphAnalyzer()
print("  ✅ Pymorphy2 готов")

# СТОП-СЛОВА
stop_words = set(stopwords.words('russian') + stopwords.words('english'))
print("  ✅ Стоп-слова загружены")

# РАСШИРЕННЫЙ СПИСОК ИГР (50+ игр)
GAMES = [
    # Популярные русскоязычные
    "Atomic Heart", "Metro Exodus", "Escape from Tarkov", "Pathfinder",
    "S.T.A.L.K.E.R.", "War Thunder", "World of Tanks", "Crossout",
    # Оригинальные
    "Hollow Knight", "Hollow Knight Silksong", "Platypus",
    "Hard Truck Apocalypse", "No Man's Sky", "Moonlighter", "Minecraft",
    # Популярные инди
    "Celeste", "Hades", "Stardew Valley", "Undertale", "Terraria",
    "Dead Cells", "Ori and the Blind Forest", "Cuphead", "Shovel Knight",
    "Subnautica", "The Binding of Isaac", "Risk of Rain 2", "Slay the Spire",
    # AAA игры
    "The Witcher 3", "Elden Ring", "Dark Souls", "Sekiro", "Bloodborne",
    "God of War", "Red Dead Redemption 2", "Cyberpunk 2077", "Skyrim",
    "Grand Theft Auto V", "The Last of Us", "Horizon Zero Dawn",
    # Стратегии
    "Civilization VI", "StarCraft II", "Age of Empires IV", "Total War",
    "XCOM 2", "Crusader Kings III", "Cities Skylines",
    # Шутеры
    "Counter-Strike 2", "Valorant", "Apex Legends", "Overwatch 2",
    "Call of Duty", "Battlefield", "Destiny 2", "Halo Infinite",
    # MMORPG/Онлайн
    "World of Warcraft", "Final Fantasy XIV", "Guild Wars 2", "Dota 2", "League of Legends",
    # Игры (расширено)
    "обзор игры", "прохождение игры", "рецензия игры", "инди игры",
    "киберспорт", "геймдизайн", "игровая индустрия", "стримы",
    "мобильные игры", "консольные игры", "PC игры", "VR игры",

    # Технологии (расширено)
    "искусственный интеллект", "машинное обучение", "программирование",
    "веб-разработка", "мобильные приложения", "кибербезопасность",
    "блокчейн технологии", "облачные вычисления", "нейросети",
    "интернет вещей", "big data", "devops", "frontend", "backend",

    # Наука (расширено)
    "космические исследования", "физика частиц", "квантовая механика",
    "биотехнологии", "генетика", "экология", "астрономия",
    "химия", "математика", "медицина", "археология", "палеонтология",

    # Культура (расширено)
    "современное искусство", "кинематограф", "литература",
    "музыкальная индустрия", "театр", "фотография", "дизайн",
    "архитектура", "выставки", "фестивали", "концерты",

    # Общество (расширено)
    "образование в России", "психология", "социология",
    "экономика", "история России", "философия", "политология",
    "право", "финансы", "маркетинг", "менеджмент", "стартапы",

    # Популярные темы (расширено)
    "новости науки", "технологии 2024", "стартапы России",
    "цифровизация", "инновации", "медицина", "здоровье",
    "экология планеты", "изменение климата", "энергетика",
    "транспорт", "урбанистика", "путешествия", "спорт", "автомобили"
]

# ФУНКЦИЯ ИЗВЛЕЧЕНИЯ АВТОРА
def extract_author(soup, url):
    """Извлекает автора статьи из HTML"""
    # Попытка 1: Мета-теги
    meta_tags = [
        soup.find('meta', {'name': 'author'}),
        soup.find('meta', {'property': 'article:author'}),
        soup.find('meta', {'name': 'article:author'}),
        soup.find('meta', {'property': 'author'}),
    ]

    for tag in meta_tags:
        if tag and tag.get('content'):
            return tag.get('content').strip()

    # Попытка 2: Класс автора
    author_classes = [
        'author', 'author-name', 'post-author', 'article-author',
        'byline', 'by-author', 'entry-author', 'writer',
        'автор', 'author_name', 'article__author'
    ]

    for class_name in author_classes:
        author_tag = soup.find(class_=re.compile(class_name, re.I))
        if author_tag:
            text = author_tag.get_text(strip=True)
            # Очистка от префиксов
            text = re.sub(r'^(by|автор|written by|posted by)[\s:]+', '', text, flags=re.I)
            if text and len(text) < 100:
                return text

    # Попытка 3: Специфичные селекторы
    selectors = [
        'a[rel="author"]',
        'span[itemprop="author"]',
        'span[itemprop="name"]',
        '[class*="author"] a',
        '.author-info a',
    ]

    for selector in selectors:
        author_tag = soup.select_one(selector)
        if author_tag:
            text = author_tag.get_text(strip=True)
            if text and len(text) < 100:
                return text

    # По умолчанию: домен сайта
    domain = urlparse(url).netloc
    return domain.replace('www.', '').split('.')[0].capitalize()

# ФУНКЦИЯ ПОИСКА ЧЕРЕЗ YANDEX (для русскоязычного контента)
def search_yandex(query, num_results=15):
    """Парсинг результатов поиска Yandex"""
    try:
        search_url = f"https://yandex.ru/search/?text={quote_plus(query)}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept-Language': 'ru-RU,ru;q=0.9'
        }
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = []
        for result in soup.find_all('a', class_=re.compile('Link.*Organic'), limit=num_results):
            href = result.get('href')
            if href and href.startswith('http'):
                links.append(href)

        # Альтернативный парсинг
        if not links:
            for result in soup.find_all('li', class_=re.compile('serp-item'), limit=num_results):
                link = result.find('a')
                if link and link.get('href'):
                    href = link.get('href')
                    if href.startswith('http'):
                        links.append(href)

        print(f"    🔍 Yandex: найдено {len(links)} ссылок для '{query}'")
        return links
    except Exception as e:
        print(f"    ⚠ Ошибка поиска Yandex: {e}")
        return []

# ФУНКЦИЯ ПОИСКА ЧЕРЕЗ DUCKDUCKGO (универсальный)
def search_duckduckgo(query, num_results=15):
    """Парсинг результатов поиска DuckDuckGo"""
    try:
        search_url = f"https://html.duckduckgo.com/html/?q={quote_plus(query)}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept-Language': 'ru-RU,ru;q=0.9,en;q=0.8'
        }
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = []
        for result in soup.find_all('a', class_='result__a', limit=num_results):
            href = result.get('href')
            if href and href.startswith('http'):
                links.append(href)

        print(f"    🔍 DuckDuckGo: найдено {len(links)} ссылок для '{query}'")
        return links
    except Exception as e:
        print(f"    ⚠ Ошибка поиска DuckDuckGo: {e}")
        return []

# СБОР БАЗОВЫХ URL (статичные источники + русскоязычные)
def get_base_urls():
    """Базовый набор проверенных источников (русскоязычные + международные)"""
    base_urls = []

    # Шаблоны для русскоязычных сайтов
    ru_templates = [
        "https://stopgame.ru/search?q={game}",
        "https://dtf.ru/search?q={game}",
        "https://igromania.ru/search/?q={game}",
        "https://www.playground.ru/search?q={game}",
        "https://www.kanobu.ru/search/?q={game}",
        "https://vgtimes.ru/search/?q={game}",
        "https://gamemag.ru/search?q={game}",
        "https://cybersport.ru/search?q={game}",
    ]

    # Шаблоны для международных сайтов
    en_templates = [
        "https://ru.wikipedia.org/wiki/{game}",
        "https://store.steampowered.com/search/?term={game}",
        "https://www.metacritic.com/search/{game}",
        "https://www.ign.com/games/{game}",
        "https://www.pcgamer.com/search/?searchTerm={game}",
        "https://www.gamespot.com/search/?q={game}",
    ]

    all_templates = ru_templates + en_templates

    for game in GAMES:
        game_slug = game.lower().replace(' ', '-').replace("'", '')
        game_encoded = quote_plus(game)

        for template in all_templates:
            if '{game}' in template:
                if 'search' in template or '?' in template:
                    url = template.format(game=game_encoded)
                else:
                    url = template.format(game=game_slug)
                base_urls.append(url)

    return base_urls

# ГЛАВНАЯ ФУНКЦИЯ СБОРА URL
def collect_urls(target_count=1500):
    """Собирает URLs из разных источников до достижения target_count"""
    print(f"\n📡 Сбор URL (цель: {target_count})...")
    all_urls = set()

    # 1. Базовые URL
    print("\n1️⃣ Добавление базовых URL...")
    base_urls = get_base_urls()
    all_urls.update(base_urls)
    print(f"  ✅ Базовых URL: {len(all_urls)}")

    # 2. Поиск через Yandex и DuckDuckGo
    print("\n2️⃣ Поиск через Yandex & DuckDuckGo...")
    search_queries = []

    # Поисковые запросы для игр (русскоязычные)
    for game in GAMES[:40]:
        search_queries.extend([
            f"{game} обзор игры",
            f"{game} прохождение",
            f"{game} рецензия",
            f"{game} статья",
        ])

    # Общие игровые темы (русскоязычные)
    general_topics = [
        "обзоры инди игр", "лучшие игры 2024", "игровые новости",
        "разработка игр", "киберспорт", "игровая культура",
        "ретро игры", "геймдизайн", "история видеоигр",
        "игровая индустрия", "игровая механика", "РПГ игры",
        "экшен игры", "стратегии", "симуляторы",
        "игровые статьи", "игровая журналистика", "обзоры новинок"
    ]
    search_queries.extend(general_topics)

    # Поиск
    for i, query in enumerate(search_queries[:800]):
        if len(all_urls) >= target_count:
            break

        print(f"  🔍 Запрос {i+1}/800: '{query}'")

        # Yandex (приоритет для русскоязычного контента)
        if random.random() > 0.3:  # 70% запросов через Yandex
            yandex_links = search_yandex(query, num_results=500)
            all_urls.update(yandex_links)
            time.sleep(random.uniform(2, 4))

        # DuckDuckGo (дополнительно)
        if len(all_urls) < target_count:
            ddg_links = search_duckduckgo(query, num_results=500)
            all_urls.update(ddg_links)
            time.sleep(random.uniform(2, 4))

        if i % 10 == 0:
            print(f"  📊 Всего собрано URL: {len(all_urls)}")

    # 3. Фильтрация нежелательных доменов
    filtered_urls = []
    blocked_domains = [
        'youtube.com', 'twitter.com', 'facebook.com', 'instagram.com',
        'reddit.com', 'discord.com', 'tiktok.com', 'pinterest.com',
        'vk.com', 'ok.ru', 't.me'  # Соц. сети
    ]

    # Приоритет русскоязычным доменам
    ru_priority_domains = [
        'stopgame.ru', 'dtf.ru', 'igromania.ru', 'playground.ru',
        'kanobu.ru', 'vgtimes.ru', 'gamemag.ru', 'cybersport.ru',
        'habr.com', 'vc.ru'
    ]

    ru_urls = []
    other_urls = []

    for url in all_urls:
        domain = urlparse(url).netloc
        if any(blocked in domain for blocked in blocked_domains):
            continue

        if any(ru_domain in domain for ru_domain in ru_priority_domains):
            ru_urls.append(url)
        else:
            other_urls.append(url)

    # Сначала русскоязычные, потом остальные
    filtered_urls = ru_urls + other_urls

    print(f"\n✅ Итоговое количество URL: {len(filtered_urls)}")
    print(f"   📌 Русскоязычных сайтов: {len(ru_urls)}")
    print(f"   🌐 Остальных сайтов: {len(other_urls)}")

    return filtered_urls[:target_count]

# HTTP SESSION
def create_http_session():
    session = requests.Session()
    ua = UserAgent()
    session.headers.update({
        'User-Agent': ua.random,
        'Accept-Language': 'ru-RU,ru;q=0.9,en;q=0.8'
    })
    return session

# ПАРСИНГ СТРАНИЦЫ
def parse_page(url, session, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=15)
            response.raise_for_status()

            # Определение кодировки
            if response.encoding.lower() in ['iso-8859-1', 'windows-1251']:
                response.encoding = response.apparent_encoding

            soup = BeautifulSoup(response.content, 'html.parser')

            # Удаляем скрипты и стили
            for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside', 'iframe']):
                tag.decompose()

            # Заголовок
            title = soup.title.string.strip() if soup.title else 'Без заголовка'

            # Автор
            author = extract_author(soup, url)

            # Контент
            content = soup.find_all(['p', 'article', 'div', 'section'], limit=150)
            raw_text = ' '.join([elem.get_text(strip=True) for elem in content if elem.get_text(strip=True)])

            if not raw_text or len(raw_text) < 200:
                return None

            # Дата публикации (попытка извлечь)
            date = datetime.now().strftime('%Y-%m-%d')
            date_meta = soup.find('meta', {'property': 'article:published_time'}) or \
                       soup.find('meta', {'name': 'date'}) or \
                       soup.find('time')

            if date_meta:
                date_str = date_meta.get('content') or date_meta.get('datetime') or date_meta.get_text()
                try:
                    date = datetime.fromisoformat(date_str.split('T')[0]).strftime('%Y-%m-%d')
                except:
                    pass

            return {
                'title': title,
                'author': author,
                'raw_text': raw_text[:10000],
                'date': date
            }
        except (requests.RequestException, Exception) as e:
            if attempt < max_retries - 1:
                time.sleep(random.uniform(1, 3))
    return None

# ОЧИСТКА И ЛЕММАТИЗАЦИЯ ТЕКСТА
def clean_text(raw_text):
    """Очистка и лемматизация русского текста"""
    # Удаление HTML
    text = re.sub(r'<[^>]+>', ' ', raw_text)
    text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)
    # Удаление чисел
    text = re.sub(r'\d+\.?\d*', ' ', text)
    # Удаление пунктуации
    text = re.sub(r'[{}]'.format(string.punctuation + '«»—–…""„'), ' ', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text.strip())

    # Токенизация
    tokens = word_tokenize(text.lower())

    # Лемматизация с pymorphy2
    lemmatized_tokens = []
    for token in tokens:
        if token.isalpha() and len(token) > 2 and token not in stop_words:
            parsed = morph.parse(token)[0]
            lemmatized_tokens.append(parsed.normal_form)

    return ' '.join(lemmatized_tokens[:400]), len(lemmatized_tokens[:400])

# ОПРЕДЕЛЕНИЕ ИГРЫ
def get_game_from_url(url, title):
    text_to_check = (url + ' ' + title).lower()
    for game in GAMES:
        if game.lower().replace(' ', '') in text_to_check.replace(' ', '').replace('-', ''):
            return game
    return "Игры (общее)"

# ОСНОВНОЙ ПРОЦЕСС
def main():
    print("\n" + "="*70)
    print("🎮 СБОРЩИК ТЕКСТОВОГО КОРПУСА - 1000+ ДОКУМЕНТОВ (РУССКОЯЗЫЧНЫЙ)")
    print("="*70)

    # Сбор URL
    target_docs = 1000
    target_urls = 1800  # Собираем больше, т.к. не все распарсятся
    unique_urls = collect_urls(target_count=target_urls)

    print(f"\n🚀 Начало парсинга {len(unique_urls)} URL (цель: {target_docs} документов)...")
    session = create_http_session()
    corpus_data = []
    parsed_urls = set()
    doc_id = 1

    random.shuffle(unique_urls)

    for i, url in enumerate(unique_urls):
        if doc_id > target_docs:
            break

        if url in parsed_urls:
            continue

        if i % 50 == 0:
            print(f"\n📊 Прогресс: {i}/{len(unique_urls)} URL | {doc_id-1} документов собрано")

        print(f"  🔍 [{doc_id}] {url[:80]}...")
        parsed = parse_page(url, session)

        if parsed and parsed['raw_text']:
            cleaned_text, token_count = clean_text(parsed['raw_text'])

            if token_count > 30:  # Минимум 30 токенов
                game = get_game_from_url(url, parsed['title'])
                corpus_data.append({
                    'doc_id': doc_id,
                    'game': game,
                    'title': parsed['title'][:200],
                    'author': parsed['author'],
                    'url': url,
                    'raw_text': parsed['raw_text'][:2000],
                    'cleaned_text': cleaned_text,
                    'tokens_count': token_count,
                    'date': parsed['date']
                })
                parsed_urls.add(url)
                print(f"    ✅ Документ {doc_id}: {token_count} токенов | Автор: {parsed['author']} | {game}")
                doc_id += 1
            else:
                print(f"    ⚠ Пропущено: мало текста ({token_count} токенов)")
        else:
            print(f"    ⚠ Пропущено: ошибка парсинга")

        time.sleep(random.uniform(0.5, 2.5))

    # СОХРАНЕНИЕ КОРПУСА В TXT
    print(f"\n💾 Создание корпуса: game_corpus_{len(corpus_data)}.txt")
    with open(f'game_corpus_{len(corpus_data)}.txt', 'w', encoding='utf-8') as f:
        for doc in corpus_data:
            f.write(f"{'='*80}\n")
            f.write(f"ДОКУМЕНТ {doc['doc_id']} | {doc['game']}\n")
            f.write(f"{'='*80}\n")
            f.write(f"Заголовок: {doc['title']}\n")
            f.write(f"Автор: {doc['author']}\n")
            f.write(f"Дата: {doc['date']}\n")
            f.write(f"Токенов: {doc['tokens_count']}\n")
            f.write(f"URL: {doc['url']}\n")
            f.write(f"{'-'*80}\n")
            f.write(f"{doc['cleaned_text']}\n")
            f.write(f"{'='*80}\n\n")
    print(f"  ✅ TXT корпус сохранён: {len(corpus_data)} документов")

    # СОХРАНЕНИЕ В CSV
    df = pd.DataFrame(corpus_data)
    df.to_csv(f'game_corpus_{len(corpus_data)}.csv', index=False, encoding='utf-8-sig')
    print(f"  ✅ CSV сохранён: game_corpus_{len(corpus_data)}.csv")

    # СТАТИСТИКА
    total_tokens = sum(doc['tokens_count'] for doc in corpus_data)
    game_counts = df['game'].value_counts()
    author_counts = df['author'].value_counts()

    print(f"\n{'='*70}")
    print(f"📊 СТАТИСТИКА КОРПУСА")
    print(f"{'='*70}")
    print(f"  📄 Документов: {len(corpus_data)}")
    print(f"  🔤 Всего токенов: {total_tokens:,}")
    print(f"  📈 Средн. токенов/документ: {total_tokens / len(corpus_data):.2f}" if corpus_data else "  Нет данных")
    print(f"  🔗 Уникальных URL: {len(parsed_urls)}")
    print(f"  ✍️  Уникальных авторов: {len(author_counts)}")

    print(f"\n🎮 Топ-10 игр по количеству документов:")
    for game, count in game_counts.head(10).items():
        print(f"    {game}: {count} документов")

    print(f"\n✍️  Топ-10 авторов:")
    for author, count in author_counts.head(10).items():
        print(f"    {author}: {count} документов")

    print(f"\n🎉 Корпус готов для NLP-анализа (LDA, кластеризация, тематическое моделирование)!")
    print(f"📁 Файлы: game_corpus_{len(corpus_data)}.txt, game_corpus_{len(corpus_data)}.csv")

if __name__ == "__main__":
    main()

🔧 Инициализация библиотек...
Загрузка ресурсов NLTK...
  ✅ NLTK готов
  ✅ Pymorphy2 готов
  ✅ Стоп-слова загружены

🎮 СБОРЩИК ТЕКСТОВОГО КОРПУСА - 1000+ ДОКУМЕНТОВ (РУССКОЯЗЫЧНЫЙ)

📡 Сбор URL (цель: 1800)...

1️⃣ Добавление базовых URL...
  ✅ Базовых URL: 1890

2️⃣ Поиск через Yandex & DuckDuckGo...

✅ Итоговое количество URL: 1890
   📌 Русскоязычных сайтов: 1080
   🌐 Остальных сайтов: 810

🚀 Начало парсинга 1800 URL (цель: 1000 документов)...

📊 Прогресс: 0/1800 URL | 0 документов собрано
  🔍 [1] https://ru.wikipedia.org/wiki/энергетика...
    ✅ Документ 1: 400 токенов | Автор: Ru | энергетика
  🔍 [2] https://stopgame.ru/search?q=Dead+Cells...
    ⚠ Пропущено: ошибка парсинга
  🔍 [2] https://www.gamespot.com/search/?q=%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D0%B8%D0...
    ✅ Документ 2: 52 токенов | Автор: Gamespot | цифровизация
  🔍 [3] https://www.ign.com/games/the-witcher-3...
    ⚠ Пропущено: ошибка парсинга
  🔍 [3] https://igromania.ru/search/?q=Overwatch+2...
    ✅ Документ 3: 400 

In [ ]:
%pip install fake_useragent
%pip install bs4
%pip install pymorphy2
%pip install spacy
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 76.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=acf2b6053b9626ed7915a3478829b2cd6a6ca13dc80f6e92cde60a3749f57b57
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


** Код скрпера страниц до проверки юез автора и со слипанием слов **

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# LIBRARIES
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
import random
import re
import string
import pandas as pd
import os
from urllib.parse import urljoin, urlparse, quote_plus
from datetime import datetime
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import spacy
import inspect

print("🔧 Initializing libraries...")

# PATCH FOR PYMORPHY2
def patch_pymorphy2():
    def getargspec_patch(func):
        try:
            args, varargs, varkw, defaults, kwonlyargs, kwonlydefaults, annotations = inspect.getfullargspec(func)
            return args, varargs, varkw, defaults
        except Exception:
            return [], None, None, None
    inspect.getargspec = getargspec_patch

patch_pymorphy2()

# NLTK SETUP
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('tokenizers/punkt_tab')
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Downloading NLTK resources...")
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
print("  ✅ NLTK ready")

# PYMORPHY2 SETUP
morph = pymorphy2.MorphAnalyzer()
print("  ✅ Pymorphy2 ready")

# SPACY SETUP
try:
    nlp = spacy.load('en_core_web_sm')
except:
    print("Downloading spaCy model: python -m spacy download en_core_web_sm")
    os.system('python -m spacy download en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')
print("  ✅ spaCy ready")

# STOP WORDS
stop_words = set(stopwords.words('english') + stopwords.words('russian'))
print("  ✅ Stop words loaded")

# РАСШИРЕННЫЙ СПИСОК ИГР (50+ игр для большего охвата)
GAMES = [
    # Оригинальные
    "Hollow Knight", "Hollow Knight Silksong", "Platypus",
    "Hard Truck Apocalypse", "No Man's Sky", "Moonlighter", "Minecraft",
    # Популярные инди
    "Celeste", "Hades", "Stardew Valley", "Undertale", "Terraria",
    "Dead Cells", "Ori and the Blind Forest", "Cuphead", "Shovel Knight",
    "Subnautica", "The Binding of Isaac", "Risk of Rain 2", "Slay the Spire",
    # AAA игры
    "The Witcher 3", "Elden Ring", "Dark Souls", "Sekiro", "Bloodborne",
    "God of War", "Red Dead Redemption 2", "Cyberpunk 2077", "Skyrim",
    "Grand Theft Auto V", "The Last of Us", "Horizon Zero Dawn",
    # Стратегии
    "Civilization VI", "StarCraft II", "Age of Empires IV", "Total War",
    "XCOM 2", "Crusader Kings III", "Cities Skylines",
    # Шутеры
    "Counter-Strike 2", "Valorant", "Apex Legends", "Overwatch 2",
    "Call of Duty", "Battlefield", "Destiny 2", "Halo Infinite",
    # MMORPG/Онлайн
    "World of Warcraft", "Final Fantasy XIV", "Guild Wars 2", "Dota 2", "League of Legends"
]

# ФУНКЦИЯ ПОИСКА ССЫЛОК ЧЕРЕЗ DUCKDUCKGO (без API ключей!)
def search_duckduckgo(query, num_results=15):
    """Парсинг результатов поиска DuckDuckGo"""
    try:
        search_url = f"https://html.duckduckgo.com/html/?q={quote_plus(query)}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = []
        for result in soup.find_all('a', class_='result__a', limit=num_results):
            href = result.get('href')
            if href and href.startswith('http'):
                links.append(href)

        print(f"    🔍 DuckDuckGo: найдено {len(links)} ссылок для '{query}'")
        return links
    except Exception as e:
        print(f"    ⚠ Ошибка поиска DuckDuckGo: {e}")
        return []

# ФУНКЦИЯ ПОИСКА ЧЕРЕЗ BING (альтернатива)
def search_bing(query, num_results=15):
    """Парсинг результатов поиска Bing"""
    try:
        search_url = f"https://www.bing.com/search?q={quote_plus(query)}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = []
        for result in soup.find_all('li', class_='b_algo', limit=num_results):
            link = result.find('a')
            if link and link.get('href'):
                href = link.get('href')
                if href.startswith('http'):
                    links.append(href)

        print(f"    🔍 Bing: найдено {len(links)} ссылок для '{query}'")
        return links
    except Exception as e:
        print(f"    ⚠ Ошибка поиска Bing: {e}")
        return []

# СБОР БАЗОВЫХ URL (статичные источники)
def get_base_urls():
    """Базовый набор проверенных источников"""
    base_urls = []

    # Шаблоны для каждой игры
    url_templates = [
        "https://en.wikipedia.org/wiki/{game}",
        "https://store.steampowered.com/search/?term={game}",
        "https://www.metacritic.com/search/{game}",
        "https://www.ign.com/games/{game}",
        "https://www.pcgamer.com/search/?searchTerm={game}",
        "https://www.gamespot.com/search/?q={game}",
        "https://www.gamesradar.com/search/?q={game}",
        "https://www.rockpapershotgun.com/?s={game}",
    ]

    for game in GAMES:
        game_slug = game.lower().replace(' ', '-').replace("'", '')
        for template in url_templates:
            url = template.format(game=game_slug)
            base_urls.append(url)

    return base_urls

# ГЛАВНАЯ ФУНКЦИЯ СБОРА URL
def collect_urls(target_count=50000):
    """Собирает URLs из разных источников до достижения target_count"""
    print(f"\n📡 Collecting URLs (target: {target_count})...")
    all_urls = set()

    # 1. Базовые URL
    print("\n1️⃣ Adding base URLs...")
    base_urls = get_base_urls()
    all_urls.update(base_urls)
    print(f"  ✅ Base URLs: {len(all_urls)}")

    # 2. Поиск через DuckDuckGo и Bing
    print("\n2️⃣ Searching via DuckDuckGo & Bing...")
    search_queries = []

    # Поисковые запросы для игр
    for game in GAMES[:30]:  # Первые 30 игр
        search_queries.extend([
            f"{game} game review",
            f"{game} gameplay",
            f"{game} wiki",
        ])

    # Общие игровые темы
    general_topics = [
        "indie game reviews", "best video games 2024", "gaming news",
        "game development", "esports", "gaming culture",
        "retro gaming", "game design", "video game history",
        "gaming industry", "game mechanics", "RPG games",
        "action games", "strategy games", "simulation games"
    ]
    search_queries.extend(general_topics)

    # Поиск
    for query in search_queries[:50]:  # Ограничение на 50 запросов
        if len(all_urls) >= target_count:
            break

        # DuckDuckGo
        ddg_links = search_duckduckgo(query, num_results=15)
        all_urls.update(ddg_links)
        time.sleep(random.uniform(0.8, 2))  # Пауза между запросами

        # Bing (для разнообразия)
        if len(all_urls) < target_count and random.random() > 0.5:
            bing_links = search_bing(query, num_results=15)
            all_urls.update(bing_links)
            time.sleep(random.uniform(0.8, 2))

        print(f"  📊 Total URLs collected: {len(all_urls)}")

    # 3. Фильтрация нежелательных доменов
    filtered_urls = []
    blocked_domains = ['youtube.com', 'twitter.com', 'facebook.com', 'instagram.com',
                      'reddit.com', 'discord.com', 'tiktok.com', 'pinterest.com']

    for url in all_urls:
        domain = urlparse(url).netloc
        if not any(blocked in domain for blocked in blocked_domains):
            filtered_urls.append(url)

    print(f"\n✅ Final URL count: {len(filtered_urls)} (filtered from {len(all_urls)})")
    return filtered_urls[:target_count]

# HTTP SESSION
def create_http_session():
    session = requests.Session()
    ua = UserAgent()
    session.headers.update({'User-Agent': ua.random})
    return session

# PARSE PAGE
def parse_page(url, session, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Удаляем скрипты и стили
            for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
                tag.decompose()

            title = soup.title.string.strip() if soup.title else 'No Title'
            content = soup.find_all(['p', 'article', 'div'], limit=100)
            raw_text = ' '.join([elem.get_text(strip=True) for elem in content if elem.get_text(strip=True)])

            if not raw_text or len(raw_text) < 150:
                return None

            date = datetime.now().strftime('%Y-%m-%d')
            return {'title': title, 'raw_text': raw_text[:8000], 'date': date}
        except (requests.RequestException, Exception) as e:
            if attempt < max_retries - 1:
                time.sleep(random.uniform(1, 3))
    return None

# CLEAN AND LEMMATIZE TEXT
def clean_text(raw_text, is_russian=False):
    text = re.sub(r'<[^>]+>', ' ', raw_text)
    text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)
    text = re.sub(r'\d+\.?\d*', ' ', text)
    text = re.sub(r'[{}]'.format(string.punctuation), ' ', text)
    text = re.sub(r'\s+', ' ', text.strip())

    tokens = word_tokenize(text.lower())
    if is_russian:
        tokens = [morph.parse(token)[0].normal_form for token in tokens
                 if token.isalpha() and token not in stop_words]
    else:
        doc = nlp(' '.join(tokens[:500]))  # Ограничение для spaCy
        tokens = [token.lemma_ for token in doc
                 if token.is_alpha and token.text not in stop_words]

    return ' '.join(tokens[:300]), len(tokens[:300])

# DETERMINE GAME
def get_game_from_url(url, title):
    text_to_check = (url + ' ' + title).lower()
    for game in GAMES:
        if game.lower().replace(' ', '') in text_to_check.replace(' ', ''):
            return game
    return "Gaming General"

# MAIN PROCESS
def main():
    print("\n" + "="*60)
    print("🎮 GAME CORPUS SCRAPER - 1000+ DOCUMENTS")
    print("="*60)

    # Сбор URL
    target_docs = 1000
    target_urls = 1500  # Собираем больше, т.к. не все распарсятся
    unique_urls = collect_urls(target_count=target_urls)

    print(f"\n🚀 Starting scraping {len(unique_urls)} URLs (target: {target_docs} docs)...")
    session = create_http_session()
    corpus_data = []
    parsed_urls = set()
    doc_id = 1

    random.shuffle(unique_urls)

    for i, url in enumerate(unique_urls):
        if doc_id > target_docs:
            break

        if url in parsed_urls:
            continue

        if i % 50 == 0:
            print(f"\n📊 Progress: {i}/{len(unique_urls)} URLs | {doc_id-1} documents collected")

        print(f"  🔍 [{doc_id}] {url[:80]}...")
        parsed = parse_page(url, session)

        if parsed and parsed['raw_text']:
            is_russian = 'ru.' in url or bool(re.search('[а-яА-Я]', parsed['raw_text'][:500]))
            cleaned_text, token_count = clean_text(parsed['raw_text'], is_russian)

            if token_count > 20:  # Минимум 20 токенов
                game = get_game_from_url(url, parsed['title'])
                corpus_data.append({
                    'doc_id': doc_id,
                    'game': game,
                    'title': parsed['title'][:150],
                    'url': url,
                    'raw_text': parsed['raw_text'][:1500],
                    'cleaned_text': cleaned_text,
                    'tokens_count': token_count,
                    'date': parsed['date']
                })
                parsed_urls.add(url)
                print(f"    ✅ Doc {doc_id}: {token_count} tokens ({game})")
                doc_id += 1
            else:
                print(f"    ⚠ Skipped: insufficient text ({token_count} tokens)")
        else:
            print(f"    ⚠ Skipped: scraping failed")

        time.sleep(random.uniform(0.5, 2))  # Пауза между запросами

    # SAVE CORPUS TO TXT
    print(f"\n💾 Creating corpus: game_corpus_{len(corpus_data)}.txt")
    with open(f'game_corpus_{len(corpus_data)}.txt', 'w', encoding='utf-8') as f:
        for doc in corpus_data:
            f.write(f"=== Document {doc['doc_id']} | {doc['game']} | {doc['title']} ===\n")
            f.write(f"URL: {doc['url']}\n")
            f.write(f"Tokens: {doc['tokens_count']} | Date: {doc['date']}\n")
            f.write(f"{doc['cleaned_text']}\n")
            f.write("="*80 + "\n\n")
    print(f"  ✅ TXT corpus saved: {len(corpus_data)} documents")

    # SAVE TO CSV
    df = pd.DataFrame(corpus_data)
    df.to_csv(f'game_corpus_{len(corpus_data)}.csv', index=False, encoding='utf-8')
    print(f"  ✅ CSV saved: game_corpus_{len(corpus_data)}.csv")

    # STATISTICS
    total_tokens = sum(doc['tokens_count'] for doc in corpus_data)
    game_counts = df['game'].value_counts()

    print(f"\n" + "="*60)
    print(f"📊 CORPUS STATISTICS")
    print("="*60)
    print(f"  📄 Documents: {len(corpus_data)}")
    print(f"  🔤 Total tokens: {total_tokens:,}")
    print(f"  📈 Avg tokens/doc: {total_tokens / len(corpus_data):.2f}" if corpus_data else "  No data")
    print(f"  🔗 Unique URLs: {len(parsed_urls)}")
    print(f"\n🎮 Top 10 games by document count:")
    for game, count in game_counts.head(10).items():
        print(f"    {game}: {count} docs")
    print(f"\n🎉 Corpus ready for NLP (LDA, clustering, topic modeling)!")

if __name__ == "__main__":
    main()

🔧 Initializing libraries...
  ✅ NLTK ready
  ✅ Pymorphy2 ready
  ✅ spaCy ready
  ✅ Stop words loaded

🎮 GAME CORPUS SCRAPER - 1000+ DOCUMENTS

📡 Collecting URLs (target: 1500)...

1️⃣ Adding base URLs...
  ✅ Base URLs: 416

2️⃣ Searching via DuckDuckGo & Bing...
    🔍 DuckDuckGo: найдено 0 ссылок для 'Hollow Knight game review'
  📊 Total URLs collected: 416
    🔍 DuckDuckGo: найдено 0 ссылок для 'Hollow Knight gameplay'
  📊 Total URLs collected: 416
    🔍 DuckDuckGo: найдено 0 ссылок для 'Hollow Knight wiki'
  📊 Total URLs collected: 416
    🔍 DuckDuckGo: найдено 0 ссылок для 'Hollow Knight Silksong game review'
    🔍 Bing: найдено 10 ссылок для 'Hollow Knight Silksong game review'
  📊 Total URLs collected: 426
    🔍 DuckDuckGo: найдено 0 ссылок для 'Hollow Knight Silksong gameplay'
    🔍 Bing: найдено 10 ссылок для 'Hollow Knight Silksong gameplay'
  📊 Total URLs collected: 436
    🔍 DuckDuckGo: найдено 0 ссылок для 'Hollow Knight Silksong wiki'
    🔍 Bing: найдено 10 ссылок для 'Holl

In [ ]:
%pip install fake_useragent

In [ ]:
%pip install fake_useragent
%pip install bs4
%pip install pymorphy2
%pip install spacy
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 67.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f568e135ae88ece9cd31cf75affc0596139f0ca388c5ddfdcdcaeae4cd02281a
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [ ]:
%pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 73.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2ed46e8e6f2047dc78dfd8b8bbcfbe6e9995d88384f0cf8e07ad305634883405
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
